In [35]:
# !brew install wget

In [36]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2022-02-22 12:58:56--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  10.0MB/s    in 16s     

2022-02-22 12:59:14 (9.29 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv        

In [37]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [39]:
# specify brackets for year as to not conflict with any films with year in title:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand = False)
# remove brackets:
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand = False)
# remove years from 'title' column:
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
# apply strip function to get rid of any ending whitespace characters that may have appeared:
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

/var/folders/dh/jzjjqv8529z6wz5qpfd7n1_r0000gn/T/ipykernel_81946/4128534981.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [40]:
# genres are split by | so use split function on |

movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [41]:
# genres are currently in list format, use One Hot Encoding technique to converts list of genres to a vector
# each column will correspond to a genre, 1 used to show valid genre, 0 used for invalid genre
# will store this dataframe in another variable as genres are not important for first rec system here

# copy movie df to new df to get rid of genres

moviesWithGenres_df = movies_df.copy()

# for every row in the df, iterate through list of genres and place a 1 into corresponding column

for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
        
# fill NaN with 0
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## ratings df

In [42]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [58]:
# drop timestamp to save memory
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

/var/folders/dh/jzjjqv8529z6wz5qpfd7n1_r0000gn/T/ipykernel_81946/1343734077.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings_df = ratings_df.drop('timestamp', 1)


KeyError: "['timestamp'] not found in axis"

## content based reccomendation system

In [59]:
# create input user to recommend movies to

userInput = [
    {'title':'Snatch', 'rating':5},
    {'title':'Casino', 'rating':3.5},
    {'title':'Inglorious Basterds', 'rating':4},
    {'title':"Pulp Fiction", 'rating':5},
    {'title':'Goodfellas', 'rating':4.5}
    ]

inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Snatch,5.0
1,Casino,3.5
2,Inglorious Basterds,4.0
3,Pulp Fiction,5.0
4,Goodfellas,4.5


In [60]:
# extract input movie ID from movies df and add them to user input df
# filter out rows that contain the movie title then merge this subset with the input df. 
# drop useless columns to save memory

inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

inputMovies = pd.merge(inputId, inputMovies)

inputMovies = inputMovies.drop('genres', 1).drop('year', 1)

inputMovies

/var/folders/dh/jzjjqv8529z6wz5qpfd7n1_r0000gn/T/ipykernel_81946/3824884591.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  inputMovies = inputMovies.drop('genres', 1).drop('year', 1)


,movieId,title,rating
0,16,Casino,3.5
1,296,Pulp Fiction,5.0
2,1213,Goodfellas,4.5
3,4011,Snatch,5.0


In [61]:
# to learn input user's preferences get a subset of movies watched from df containing binary values

userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
15,16,Casino,"[Crime, Drama]",1995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1187,1213,Goodfellas,"[Crime, Drama]",1990,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3918,4011,Snatch,"[Comedy, Crime, Thriller]",2000,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
# only the genre table is needed. clean up by resetting index and dropping movieId, title, genres & year columns

userMovies = userMovies.reset_index(drop=True)

userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

/var/folders/dh/jzjjqv8529z6wz5qpfd7n1_r0000gn/T/ipykernel_81946/1990713031.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# to start learning input user's preferences:
# turn genres into weights by multiplying users reviews into genres table then sum up each column 
# (this is a dot product between matrix and vector)

inputMovies['rating']


0    3.5
1    5.0
2    4.5
3    5.0
Name: rating, dtype: float64

In [64]:
# dot product to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

Adventure              0.0
Animation              0.0
Children               0.0
Comedy                10.0
Fantasy                0.0
Romance                0.0
Drama                 13.0
Action                 0.0
Crime                 18.0
Thriller              10.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 0.0
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [65]:
# extract genre table from original dataframe

genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
# drop useless columns
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

/var/folders/dh/jzjjqv8529z6wz5qpfd7n1_r0000gn/T/ipykernel_81946/812620389.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
genreTable.shape

In [66]:
# with the input users profile and complete list of movies & genres,  take weighted average of every movie based
# on input profile and recommend top 20 that most satisfy it

# multiply genres by weights and then take weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.196078
2    0.000000
3    0.196078
4    0.450980
5    0.196078
dtype: float64

In [67]:
# sort in desc. order

recommendationTable_df = recommendationTable_df.sort_values(ascending = False)
recommendationTable_df.head()

movieId
1912      1.0
57669     1.0
115333    1.0
115335    1.0
115337    1.0
dtype: float64

In [68]:
# final top 20 recs:

movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
1726,1799,Suicide Kings,"[Comedy, Crime, Drama, Mystery, Thriller]",1997
1829,1912,Out of Sight,"[Comedy, Crime, Drama, Romance, Thriller]",1998
3801,3893,Nurse Betty,"[Comedy, Crime, Drama, Romance, Thriller]",2000
4932,5027,Another 48 Hrs.,"[Action, Comedy, Crime, Drama, Thriller]",1990
8613,26104,Murder at the Gallop,"[Comedy, Crime, Drama, Mystery, Thriller]",1963
9226,27171,Freeway II: Confessions of a Trickbaby,"[Comedy, Crime, Drama, Thriller]",1999
10852,43853,"Business, The","[Action, Comedy, Crime, Drama, Thriller]",2005
11773,52202,His Kind of Woman,"[Comedy, Crime, Drama, Film-Noir, Thriller]",1951
12015,54262,Murder Most Foul,"[Comedy, Crime, Drama, Mystery, Thriller]",1964
12427,57669,In Bruges,"[Comedy, Crime, Drama, Thriller]",2008
